In [17]:
import numpy as np
from scipy.stats import norm
import math

In [18]:
def binomial_option_pricing(S0,K,T,r,sigma,N,option_type='call'):
    """
    European option price via binomial model
    
    Parameters:
    S0 : Initial asset price (float)
    K : Strike price (float)
    T : Maturity time in years (float)
    r : Risk free interest rate (float)
    sigma : Volatility of the asset (float)
    N : Number of steps in the binomial tree model (int)
    option_type : Type of the option 'call' or 'put' (str)
    
    Returns: The price of the option (float)
    """

    # time step
    dt=T/N
    # up and down factors
    u=np.exp(sigma * np.sqrt(dt))
    d=1/u
    # risk-neutral probability
    p=(np.exp(r*dt)-d)/(u-d)
    # computing asset prices and corresponding probabilities at the last step
    asset_prices=np.array([])
    asset_prob=np.array([])
    for i in range(0, N + 1):
        asset_prices=np.append(asset_prices,S0*u**(N-i)*d**i)
        asset_prob=np.append(asset_prob,math.comb(N,i)* p**(N-i)*(1-p)**i)

    # net option values at last step
    if option_type=='call':
        option_values=np.maximum(0,asset_prices-K)
    elif option_type=='put':
        option_values=np.maximum(0,K-asset_prices)

    # option price
    bi_option_value=np.exp(-r*T)*np.dot(option_values,asset_prob)
    return bi_option_value

In [19]:
def monte_carlo_option_pricing(S0,K,T,r,sigma,N,num_sim,option_type='call'):
    """
    European option price via Monte Carlo method

    Parameters:
    S0: Initial stock price (float)
    K: Strike price (float)
    T: Time to maturity (float)
    r: Risk free interest rate (float)
    sigma: Volatility of the asset (float)
    N: Number of time steps (int)
    num_sim: Number of simulations (int)
    option_type: Type of the option 'call' or 'put' (str)

    Returns: The price of the option (float)
    """
    # time step
    dt=T/N
    # simulation of the asset price
    asset_path=np.zeros((num_sim,N+1))
    asset_path[:,0]=S0
    for i in range(1,N+1):
        Z=np.random.standard_normal(num_sim)
        asset_path[:,i]=asset_path[:,i-1]*np.exp((r-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z)
    if option_type=='call':
        payoffs=np.maximum(asset_path[:,-1]-K,0)
    elif option_type=='put':
        payoffs=np.maximum(K-asset_path[:,-1],0)

    # option price
    mc_option_value=np.exp(-r*T)*np.mean(payoffs)
    return mc_option_value

In [20]:
def black_scholes_option_pricing(S0,K,T,r,sigma,option_type='call'):
    """
    European option price via Black Scholes model

    Parameters:
    S0: Initial stock price (float)
    K: Strike price (float)
    T: Time to maturity (float)
    r: Risk free interest rate (float)
    sigma: Volatility of the asset (float)
    option_type: Type of the option 'call' or 'put' (str)

    Returns: The price of the option (float)
    """
    
    # calculate d1 and d2
    d1=(np.log(S0/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2=d1-sigma*np.sqrt(T)
    
    # option value
    if option_type=='call':
        bs_option_value=S0*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
    elif option_type=='put':
        bs_option_value=K*np.exp(-r*T)*norm.cdf(-d2)-S0*norm.cdf(-d1)
    
    return bs_option_value

In [23]:
# An example
S0=120   # Initial stock price
K=130    # Strike price
T=0.5      # Maturity time (6 months)
r=0.05   # Risk-free interest rate (5% per annum)
sigma=0.15  # Volatility (15% per annum)
N=500    # Number of steps
num_sim=10000 # Number of simulations

bi_call_price=binomial_option_pricing(S0,K,T,r,sigma,N,option_type='call')
bi_put_price = binomial_option_pricing(S0,K,T,r,sigma,N,option_type='put')

mc_call_price=monte_carlo_option_pricing(S0,K,T,r,sigma,N,num_sim,option_type='call')
mc_put_price=monte_carlo_option_pricing(S0,K,T,r,sigma,N,num_sim,option_type='put')

bs_call_price=black_scholes_option_pricing(S0,K,T,r,sigma,option_type='call')
bs_put_price=black_scholes_option_pricing(S0,K,T,r,sigma,option_type='put')

print(f"Binomial model: Call option price: {bi_call_price:.3f} and Put option price: {bi_put_price:.3f}")
print(f"Monte Carlo method: Call option price: {mc_call_price:.3f} and Put option price: {mc_put_price:.3f}")
print(f"Black Scholes model: Call option price: {bs_call_price:.3f} and Put option price: {bs_put_price:.3f}")

Binomial model: Call option price: 2.512 and Put option price: 9.302
Monte Carlo method: Call option price: 2.509 and Put option price: 9.335
Black Scholes model: Call option price: 2.510 and Put option price: 9.301
